In [ ]:

"""import pandas as pd

def berechne_immobilienpreis(df):
    # Annahme: Basispreis pro m² Wohnfläche (abhängig von der Stadt/Land Lage)
    basispreis_stadt = 3000  # €/m²
    basispreis_land = 2000   # €/m²
    
    preise = []
    
    for _, row in df.iterrows():
        wohnflaeche = row['Wohnfläche (m²)']
        grundstueck = row['Grundstück (m²)']
        architektenhaus = row['Architektenhaus']
        makler = row['Makler']
        denkmalschutz = row['Denkmalschutz']
        baujahr = row['Baujahr']
        lage = row['Lage']
        ausstattung = row['Ausstattung']
        hausart = row['Hausart']
        bundesland = row['Bundesland']
        
        # Basispreis bestimmen
        basispreis = basispreis_stadt if lage == "Stadt" else basispreis_land
        
        # Altersabschlag
        altersabschlag = max(0, (2025 - baujahr) * 0.5)
        
        # Zuschläge und Abschläge
        zuschlag_architekt = 1.2 if architektenhaus else 1.0
        abschlag_makler = 0.95 if makler else 1.0
        zuschlag_denkmalschutz = 1.1 if denkmalschutz else 1.0
        
        # Ausstattungsfaktor (1 = einfach, 1.2 = gehoben, 1.5 = luxuriös)
        ausstattungsfaktor = 1.0 if ausstattung == "einfach" else 1.2 if ausstattung == "gehoben" else 1.5
        
        # Berechnung des Preises
        preis = wohnflaeche * basispreis
        preis *= zuschlag_architekt * abschlag_makler * zuschlag_denkmalschutz * ausstattungsfaktor
        preis -= altersabschlag * 1000  # Abschlag basierend auf Alter
        
        # Grundstückswert (pauschal 300 €/m²)
        preis += grundstueck * 300
        
        preise.append(preis)
    
    df['Geschätzter Preis (€)'] = preise
    return df

# Beispiel-Daten laden
daten = {
    'Wohnfläche (m²)': [120, 150],
    'Grundstück (m²)': [500, 800],
    'Architektenhaus': [True, False],
    'Makler': [False, True],
    'Denkmalschutz': [False, True],
    'Baujahr': [2000, 1950],
    'Lage': ["Stadt", "Land"],
    'Ausstattung': ["gehoben", "einfach"],
    'Hausart': ["Einfamilienhaus", "Villa"],
    'Bundesland': ["Bayern", "NRW"]
}

df = pd.DataFrame(daten)
df = berechne_immobilienpreis(df)

# Anzeige der berechneten Werte
import ace_tools as tools

tools.display_dataframe_to_user(name="Immobilienpreis Berechnung", dataframe=df)"""

In [9]:
import tkinter as tk
from tkinter import messagebox, ttk
import pandas as pd

# Laden der Preisfaktoren aus der Excel-Datei
def lade_preisdaten(dateipfad):
    xls = pd.ExcelFile(dateipfad)
    df = pd.read_excel(xls, sheet_name='Sheet1')
    return df

# Berechnung des Immobilienpreises
def berechne_preis():
    try:
        bundesland = combo_bundesland.get()
        region = combo_region.get()
        ausstattung = combo_ausstattung.get()
        hausart = combo_hausart.get()
        wohnflaeche = float(entry_wohnflaeche.get())
        grundstueck = float(entry_grundstueck.get())
        baujahr = int(entry_baujahr.get())
        architektenhaus = var_architektenhaus.get()
        makler = var_makler.get()
        denkmalschutz = var_denkmalschutz.get()

        # Beispielwerte, diese sollten aus der Excel-Datei kommen
        preis_pro_m2_wohnflaeche = 2500  # €/m²
        preis_pro_m2_grundstueck = 160   # €/m²

        # Anpassung nach Bundesland
        bundesland_faktoren = {
            "Nordrhein-Westfalen": 1.1,
            "Bayern": 1.2,
            "Berlin": 1.3,
            "Hamburg": 1.25
        }
        faktor_bundesland = bundesland_faktoren.get(bundesland, 1.0)

        # Anpassung nach Region
        faktor_region = 1.2 if region == "Stadt" else 1.0

        # Anpassung nach Ausstattung
        ausstattungs_faktoren = {
            "Einfach": 0.9,
            "Gehoben": 1.1,
            "Luxuriös": 1.3
        }
        faktor_ausstattung = ausstattungs_faktoren.get(ausstattung, 1.0)

        # Altersabschlag berechnen
        altersabschlag = max(0, (2025 - baujahr) * 0.005)  # 0.5% pro Jahr

        # Zuschläge/Abschläge
        zuschlag_architektenhaus = 1.2 if architektenhaus else 1.0
        zuschlag_denkmalschutz = 1.1 if denkmalschutz else 1.0

        # Berechnung des Gesamtpreises
        gesamtpreis = (wohnflaeche * preis_pro_m2_wohnflaeche) * (1 - altersabschlag)
        gesamtpreis += grundstueck * preis_pro_m2_grundstueck
        gesamtpreis *= (faktor_bundesland * faktor_region * faktor_ausstattung)
        gesamtpreis *= zuschlag_architektenhaus * zuschlag_denkmalschutz

        # Maklergebühr berücksichtigen
        maklergebuehr = 0.0357 * gesamtpreis if makler else 0
        gesamtpreis += maklergebuehr

        messagebox.showinfo("Ergebnis", f"Der geschätzte Preis beträgt: {gesamtpreis:,.2f} € (inkl. Maklergebühr: {maklergebuehr:,.2f} €)")
    except ValueError:
        messagebox.showerror("Fehler", "Bitte geben Sie gültige Zahlen ein.")

# GUI erstellen
root = tk.Tk()
root.title("Immobilienpreis Rechner")

# Labels und Eingabefelder
tk.Label(root, text="Bundesland:").grid(row=0, column=0)
combo_bundesland = ttk.Combobox(root, values=["Nordrhein-Westfalen", "Bayern", "Berlin", "Hamburg"])
combo_bundesland.grid(row=0, column=1)

tk.Label(root, text="Region:").grid(row=1, column=0)
combo_region = ttk.Combobox(root, values=["Stadt", "Land"])
combo_region.grid(row=1, column=1)

tk.Label(root, text="Ausstattung:").grid(row=2, column=0)
combo_ausstattung = ttk.Combobox(root, values=["Einfach", "Gehoben", "Luxuriös"])
combo_ausstattung.grid(row=2, column=1)

tk.Label(root, text="Hausart:").grid(row=3, column=0)
combo_hausart = ttk.Combobox(root, values=["Einfamilienhaus", "Doppelhaushälfte", "Villa"])
combo_hausart.grid(row=3, column=1)

tk.Label(root, text="Grundstücksgröße (m²):").grid(row=4, column=0)
entry_grundstueck = tk.Entry(root)
entry_grundstueck.grid(row=4, column=1)

tk.Label(root, text="Wohnfläche (m²):").grid(row=5, column=0)
entry_wohnflaeche = tk.Entry(root)
entry_wohnflaeche.grid(row=5, column=1)

tk.Label(root, text="Baujahr:").grid(row=6, column=0)
entry_baujahr = tk.Entry(root)
entry_baujahr.grid(row=6, column=1)

var_architektenhaus = tk.BooleanVar()
chk_architektenhaus = tk.Checkbutton(root, text="Architektenhaus", variable=var_architektenhaus)
chk_architektenhaus.grid(row=7, column=0, columnspan=2)

var_makler = tk.BooleanVar()
chk_makler = tk.Checkbutton(root, text="Makler", variable=var_makler)
chk_makler.grid(row=8, column=0, columnspan=2)

var_denkmalschutz = tk.BooleanVar()
chk_denkmalschutz = tk.Checkbutton(root, text="Denkmalschutz", variable=var_denkmalschutz)
chk_denkmalschutz.grid(row=9, column=0, columnspan=2)

# Berechnungsbutton
tk.Button(root, text="Berechnen", command=berechne_preis).grid(row=10, column=0, columnspan=2)

# Start der GUI
root.mainloop()

# Noch nicht fertig!!!